# Preliminary Steps

The first step of this project is to load the data correctly and check for any missing values. The data is stored in `data` folder.

The given excel workbooks contain a sheet named `data`, which comprises of the data of the respective suburbs. 

The leftmost column (as seen when loaded in Microsoft Excel) describes the category of the features- we'll be referring this column as the 'metadata' column. 

Now, as the data is given in its original format, the 'metadata' column only contains a few blocks, with each block aligned against multiple feature-names to its right which fall into the same category. To adjust this, we have repeated the entries of the metadata column to resolve this entry-mismatch issue and created a single `test.xlsx` file. The first column of this file will be later used to replace the metadata column of the others.

In [1]:
import pandas as pd
import numpy as np

df_test = pd.read_excel('test.xlsx', sheet_name='data', header=None)

df_test

,0,1,2
0,Community,Community Name,Braybrook (Suburb)
1,Community,Region,Northern and Western Metropolitan
2,Geography,Map reference,4
3,Geography,Grid reference,A3
4,Geography,Location,10km WNW of Melbourne
...,...,...,...
221,Hospital,Distance to nearest public hospital with emerg...,10.161988
222,Hospital,Presentations to emergency departments due to ...,543.631989
223,Hospital,Presentations to emergency departments due to ...,20.647263
224,Hospital,Category 4 & 5 emergency department presentations,1683.966712


We can now create a reference column that we'll be using to replace the first columns of the other sheets. 

A dictionary is also created for looking up the feature-names that belong to the same category, with the categories as the keys.

In [2]:
reference_column=df_test.iloc[:,0]
feature_category_dict=df_test.groupby(0)[1].unique().to_dict()

## Loading the excel sheets

We first check if the number of features across the datasets is identical. If yes, then we load all the datasets into memory and adjust for downstream tasks.

In [3]:
# utility function to load the excel files
def excel_loader(xlsx_path, sheet_name='data', ref_col=reference_column):
	# load the excel file
	df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None)
	# replace the first column with the reference column
	df.iloc[:, 0] = ref_col
	# change the column names
	df.columns = ['feature_kind', 'feature_name', 'feature_value']
	return df

# get the list of sheets

import os
dataset_list=[i for i in os.listdir('./data/') if i.endswith('.xlsx')]

# sort the list
dataset_list.sort()


len_list=[]
feature_num_consistency = True
for i in dataset_list:
	len_list.append(len(pd.read_excel('./data/'+i, sheet_name='data', header=None)[1]))
		

# check if the number of features is consistent across all the datasets
if all(x == len_list[0] for x in len_list):
	print('The number of features is consistent across all the datasets')
	print('The number of features is:', len_list[0])
else:
	print('The number of features is not consistent across all the datasets')
	feature_num_consistency = False
	print('The number of features in each dataset is:', len_list)

# load and adjust the excel files
df_all = [excel_loader('./data/'+i) for i in dataset_list]

The number of features is consistent across all the datasets
The number of features is: 226


### Checking which sheets contain missing values

In [4]:
# check for missing values in any of the datasets
missing_values = [df.isnull().values.any() for df in df_all]
# print the index of the dataset with missing values
dfs_with_na=[i for i, x in enumerate(missing_values) if x]

print('Datasets with missing values:')
for i in dfs_with_na:
	print(f'Index: {i}, Dataset: {dataset_list[i]}')

Datasets with missing values:
Index: 8, Dataset: Malvern-Suburb - XLSX.xlsx
Index: 9, Dataset: Melbourne-Airport-Suburb - XLSX.xlsx
Index: 22, Dataset: Sorrento-Suburb - XLSX.xlsx
Index: 26, Dataset: St-Andrews-Beach-Suburb - XLSX.xlsx
Index: 29, Dataset: St-Kilda-West-Suburb - XLSX.xlsx
Index: 30, Dataset: Toorak-Suburb - XLSX.xlsx
Index: 31, Dataset: Tyabb-Suburb - XLSX.xlsx
Index: 32, Dataset: Waterways-Suburb - XLSX.xlsx


In [5]:
# write utility function to get the index and the feature_category, feature_name, feature_value of the missing values for a given dataset

def get_missing_values(df):
	# get the index of the missing values
	missing_values_index = df[df.isnull().any(axis=1)].index
	# get the feature_category, feature_name, feature_value of the missing values
	missing_values = df.iloc[missing_values_index]
	return missing_values


affected_details = [get_missing_values(df_all[i]) for i in dfs_with_na]

affected_rows_count=[df.shape[0] for df in affected_details]

print('The number of rows with missing values in each dataset:')
for i in range(len(affected_rows_count)):
	print(f'Index: {dfs_with_na[i]}, Dataset: {dataset_list[dfs_with_na[i]]}, Rows: {affected_rows_count[i]}')



The number of rows with missing values in each dataset:
Index: 8, Dataset: Malvern-Suburb - XLSX.xlsx, Rows: 1
Index: 9, Dataset: Melbourne-Airport-Suburb - XLSX.xlsx, Rows: 52
Index: 22, Dataset: Sorrento-Suburb - XLSX.xlsx, Rows: 5
Index: 26, Dataset: St-Andrews-Beach-Suburb - XLSX.xlsx, Rows: 8
Index: 29, Dataset: St-Kilda-West-Suburb - XLSX.xlsx, Rows: 1
Index: 30, Dataset: Toorak-Suburb - XLSX.xlsx, Rows: 1
Index: 31, Dataset: Tyabb-Suburb - XLSX.xlsx, Rows: 1
Index: 32, Dataset: Waterways-Suburb - XLSX.xlsx, Rows: 5


### Combining the datasets into one

In [6]:
# Combining the data from all the datasets
# modified temporary dataframes will have 'feature_name' and 'feature_value' columns

modified_dfs = [df[['feature_name', 'feature_value']] for df in df_all]

# modify each element of the modified_dfs list
# so that it is transposed and the first row is the feature_name and the second row is the feature_value
modified_dfs = [df.T for df in modified_dfs]

# set the first row as the column names
modified_dfs = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in modified_dfs]

# # add the suburb column to each dataframe
# for i, df in enumerate(modified_dfs):
# 	df['suburb'] = suburb_dict[i]

# concatenate all the dataframes
combined_df = pd.concat(modified_dfs)

# reset the index
combined_df.reset_index(drop=True, inplace=True)


combined_df

,Community Name,Region,Map reference,Grid reference,Location,Population Density,Travel time to GPO (minutes),Distance to GPO (km),LGA,Primary Care Partnership,...,Time to nearest public hospital with maternity services,Distance to nearest public hospital with maternity services,"Presentations to emergency departments, 2012-13",Nearest public hospital with emergency department,Travel time to nearest public hospital with emergency department,Distance to nearest public hospital with emergency department,Presentations to emergency departments due to injury,"Presentations to emergency departments due to injury, %",Category 4 & 5 emergency department presentations,"Category 4 & 5 emergency department presentations, %"
0,Ascot Vale (Suburb),Northern and Western Metropolitan,4,B3,6km NW of Melbourne,3758.623596,9.360142,6.958742,Moonee Valley (C),Inner North West Primary Care Partnership,...,6.490453,4.91257,3313.05218,Royal Melbourne Hospital,6.630953,4.993841,679.257076,20.502456,1864.918123,56.290032
1,Braybrook (Suburb),Northern and Western Metropolitan,4,A3,10km WNW of Melbourne,2025.468296,15.131666,11.595888,Maribyrnong (C),"HealthWest (b): Hobson's Bay, Maribyrnong, Wyn...",...,8.071881,6.216803,2632.949379,Royal Melbourne Hospital,12.824977,10.161988,543.631989,20.647263,1683.966712,63.957428
2,Craigieburn (Suburb),Northern and Western Metropolitan,2,A3,27km N of Melbourne,1034.97087,31.994666,43.100287,Hume (C),Hume-Whittlesea Primary Care Partnership,...,11.570855,15.213189,9915.723721,The Northern Hospital,11.570855,15.213189,2044.424399,20.618005,5102.134434,51.454988
3,Croydon (Suburb),Eastern Metropolitan,2,B4,28km E of Melbourne,1730.06483,28.992647,34.071323,Maroondah (C),Outer East Primary Care Partnership,...,10.683462,9.413847,6149.574954,Maroondah Hospital,5.093285,3.601752,1754.954941,28.537825,3062.182462,49.795026
4,Fawkner (Suburb),Northern and Western Metropolitan,4,C1,12km N of Melbourne,2619.120089,17.405267,13.047142,Moreland (C),Inner North West Primary Care Partnership,...,11.510757,12.004044,3799.03089,The Northern Hospital,11.510757,12.004044,680.401318,17.909865,1942.874353,51.141315
5,Footscray (Suburb),Northern and Western Metropolitan,4,B3,6km WNW of Melbourne,2848.743284,9.394512,7.034739,Maribyrnong (C),"HealthWest (b): Hobson's Bay, Maribyrnong, Wyn...",...,7.392323,5.651567,4252.700131,Royal Melbourne Hospital,7.261573,5.514839,878.066191,20.647263,2719.917638,63.957428
6,Glenroy (Suburb),Northern and Western Metropolitan,4,B1,13km NNW of Melbourne,2352.122818,15.982083,14.687751,Moreland (C),Inner North West Primary Care Partnership,...,12.431076,14.722128,5937.063365,Royal Melbourne Hospital,13.252894,12.72285,1063.320056,17.909865,3036.292274,51.141315
7,Malvern East (Suburb),Southern Metropolitan,1,D2,11km SE of Melbourne,2795.946571,13.811856,13.962712,Stonnington (C),Inner South East Partnership in Community & He...,...,8.746589,8.006951,1653.697142,Monash Medical Centre (Clayton),8.746589,8.006951,324.067452,19.596542,847.427721,51.244433
8,Malvern (Suburb),Southern Metropolitan,1,C2,8km SE of Melbourne,3686.421646,11.983264,11.583241,Stonnington (C),Inner South East Partnership in Community & He...,...,13.550653,12.26548,792.22707,The Alfred,7.386301,5.312177,155.249109,19.596542,405.972268,51.244433
9,Melbourne Airport (Suburb),Northern and Western Metropolitan,1,E4,19km NW of Melbourne,5.200815,22.65084,24.572985,Hume (C),Hume-Whittlesea Primary Care Partnership,...,13.488272,13.527362,42.865285,Royal Melbourne Hospital,19.921651,22.608084,8.837967,20.618005,22.056327,51.454988


In [7]:
# drop the trailing '(Suburb)' from the community names

combined_df['Community Name'] = [i.split(' (Suburb)')[0] for i in combined_df['Community Name']]


combined_df


,Community Name,Region,Map reference,Grid reference,Location,Population Density,Travel time to GPO (minutes),Distance to GPO (km),LGA,Primary Care Partnership,...,Time to nearest public hospital with maternity services,Distance to nearest public hospital with maternity services,"Presentations to emergency departments, 2012-13",Nearest public hospital with emergency department,Travel time to nearest public hospital with emergency department,Distance to nearest public hospital with emergency department,Presentations to emergency departments due to injury,"Presentations to emergency departments due to injury, %",Category 4 & 5 emergency department presentations,"Category 4 & 5 emergency department presentations, %"
0,Ascot Vale,Northern and Western Metropolitan,4,B3,6km NW of Melbourne,3758.623596,9.360142,6.958742,Moonee Valley (C),Inner North West Primary Care Partnership,...,6.490453,4.91257,3313.05218,Royal Melbourne Hospital,6.630953,4.993841,679.257076,20.502456,1864.918123,56.290032
1,Braybrook,Northern and Western Metropolitan,4,A3,10km WNW of Melbourne,2025.468296,15.131666,11.595888,Maribyrnong (C),"HealthWest (b): Hobson's Bay, Maribyrnong, Wyn...",...,8.071881,6.216803,2632.949379,Royal Melbourne Hospital,12.824977,10.161988,543.631989,20.647263,1683.966712,63.957428
2,Craigieburn,Northern and Western Metropolitan,2,A3,27km N of Melbourne,1034.97087,31.994666,43.100287,Hume (C),Hume-Whittlesea Primary Care Partnership,...,11.570855,15.213189,9915.723721,The Northern Hospital,11.570855,15.213189,2044.424399,20.618005,5102.134434,51.454988
3,Croydon,Eastern Metropolitan,2,B4,28km E of Melbourne,1730.06483,28.992647,34.071323,Maroondah (C),Outer East Primary Care Partnership,...,10.683462,9.413847,6149.574954,Maroondah Hospital,5.093285,3.601752,1754.954941,28.537825,3062.182462,49.795026
4,Fawkner,Northern and Western Metropolitan,4,C1,12km N of Melbourne,2619.120089,17.405267,13.047142,Moreland (C),Inner North West Primary Care Partnership,...,11.510757,12.004044,3799.03089,The Northern Hospital,11.510757,12.004044,680.401318,17.909865,1942.874353,51.141315
5,Footscray,Northern and Western Metropolitan,4,B3,6km WNW of Melbourne,2848.743284,9.394512,7.034739,Maribyrnong (C),"HealthWest (b): Hobson's Bay, Maribyrnong, Wyn...",...,7.392323,5.651567,4252.700131,Royal Melbourne Hospital,7.261573,5.514839,878.066191,20.647263,2719.917638,63.957428
6,Glenroy,Northern and Western Metropolitan,4,B1,13km NNW of Melbourne,2352.122818,15.982083,14.687751,Moreland (C),Inner North West Primary Care Partnership,...,12.431076,14.722128,5937.063365,Royal Melbourne Hospital,13.252894,12.72285,1063.320056,17.909865,3036.292274,51.141315
7,Malvern East,Southern Metropolitan,1,D2,11km SE of Melbourne,2795.946571,13.811856,13.962712,Stonnington (C),Inner South East Partnership in Community & He...,...,8.746589,8.006951,1653.697142,Monash Medical Centre (Clayton),8.746589,8.006951,324.067452,19.596542,847.427721,51.244433
8,Malvern,Southern Metropolitan,1,C2,8km SE of Melbourne,3686.421646,11.983264,11.583241,Stonnington (C),Inner South East Partnership in Community & He...,...,13.550653,12.26548,792.22707,The Alfred,7.386301,5.312177,155.249109,19.596542,405.972268,51.244433
9,Melbourne Airport,Northern and Western Metropolitan,1,E4,19km NW of Melbourne,5.200815,22.65084,24.572985,Hume (C),Hume-Whittlesea Primary Care Partnership,...,13.488272,13.527362,42.865285,Royal Melbourne Hospital,19.921651,22.608084,8.837967,20.618005,22.056327,51.454988


In [8]:
# write the combined data to csv
combined_df.to_csv('combined_data.csv', index=False)


## Saving the category mapping to a json file

In [9]:
# save feature_category_dict as a json file

feature_category_dict_to_list={k: list(v) for k, v in feature_category_dict.items()}

import json
with open('feature_category_dict.json', 'w') as f:
	json.dump(feature_category_dict_to_list, f)

	

In [11]:
# load the json file to check if it was saved correctly
with open('feature_category_dict.json', 'r') as f:
	data = json.load(f)

data['Geography']

['Map reference',
 'Grid reference',
 'Location',
 'Population Density',
 'Travel time to GPO (minutes)',
 'Distance to GPO (km)',
 'LGA',
 'Primary Care Partnership',
 'Medicare Local',
 'Area (km^2)',
 'ARIA+ (min)',
 'ARIA+ (max)',
 'ARIA+ (avg)',
 'ABS remoteness category',
 'DHS Area']